In [1]:
# 전처리 1-3, 앵커 목소리가 기자 목소리로 전환될 때 timestamp 남기기

# 참고 자료 : https://github.com/pyannote/pyannote-audio/tree/master/tutorials/pretrained/model

# [ 고민 지점 ]
# 연산 처리 시간이 생각보다 오래 걸리는데 이걸 어떻게 극복해야 할까?

In [2]:
# !pip install git+https://github.com/pyannote/pyannote-audio

In [3]:
import torch
import pyannote
from pyannote.database import get_protocol
from pyannote.database import FileFinder
from pyannote.audio.utils.signal import Binarize

import sys
import os
import re
import moviepy.editor as mp
import numpy as np
import pandas as pd
import datetime
import time

C:\Users\YangGang\anaconda3\lib\site-packages\pyannote\database\database.py:51: UserWarning: Ignoring deprecated 'preprocessors' argument in AMI.__init__. Pass it to 'get_protocol' instead.
  warnings.warn(msg)


In [4]:
# 코드 실행시간 측정
start = time.time()

In [5]:
# 총정리할 때 전처리 1-1 함수의 반환값으로 input 인수값 변경
# (전처리 1-1 함수의 반환값 : 앵커 1명의 얼굴로 시작되는 영상 리스트)

# MOV_to_WAV 함수 : 영상(.mp4) -> 음성(.wav) 전환 작업
def MOV_to_WAV(MOV_Directory):
    
    VideoList = os.listdir(MOV_Directory) 
    WAV_Directory = './WAV'

    for i in VideoList:
        i2 = MOV_Directory + '/' + i
        clip = mp.VideoFileClip(i2)

        sound_file_name = re.sub(pattern='mp4', repl='wav', string=i)
        sound_file_name = WAV_Directory + '/' +sound_file_name
        clip.audio.write_audiofile(sound_file_name)

In [6]:
# WAV_file_name 함수 : WAV 파일 array 반환
def WAV_file_name(WAV_Directory):
    SoundList = os.listdir(WAV_Directory)
    WAV = []

    for i in SoundList:
        i2 = WAV_Directory + '/' + i
        WAV.append(i2)

    WAV = np.array(WAV)
    
    return WAV

In [7]:
# Using_sad 함수 : Speech activity detection (sad) 사전훈련 모델 사용 선언 및 설정
# (+ WAV_file_name 함수의 반환값을 인수로 넣기)
def Using_sad(WAV):
    sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')
    preprocessors = {'audio': FileFinder()}
    protocol = get_protocol('AMI.SpeakerDiarization.MixHeadset',
                            preprocessors=preprocessors)
    
    # Speech activity detection 환경 설정
    binarize = Binarize(offset=0.52, onset=0.52, log_scale=True,
                       min_duration_off=0.1, min_duration_on=0.1)

    # 앵커 -> 기자 넘어가는 시점 계산
    test_file = {}
    results = pd.DataFrame({'WAV_name':[], 'TimeStamp':[]})
    
    for k in WAV:
        test_file['audio'] = k
        sad_scores = sad(test_file)
        speech = binarize.apply(sad_scores, dimension=1)
    #     print(test_file['audio'], str(speech[0]))

        a = str(speech[0])
        a = re.sub(pattern='\-\-\>', repl='', string=a)
        a = re.sub(pattern='\[', repl='', string=a)
        a = re.sub(pattern='\]', repl='', string=a)
        a = str.split(a)
        
        b = a[1]
        b = datetime.datetime.strptime(b, '%H:%M:%S.%f').time()
        result = pd.DataFrame({'WAV_name':[test_file['audio']], 'TimeStamp':[b]})
        
        results = pd.concat([results, result])
    
    return results

In [8]:
# Switching_moment 함수 : 전처리 1-3 파트 총망라
def Switching_moment():
    # mp4 -> wav 파일 변환
    MOV_to_WAV(MOV_Directory = './MOV')
    
    # wav 파일 이름 구하기
    WAV_files = WAV_file_name(WAV_Directory = './WAV')
    
    # 앵커 -> 기자 넘어가는 시점 계산
    result_table = Using_sad(WAV=WAV_files)
    
    return result_table

In [9]:
Switching_moment()

MoviePy - Writing audio in ./WAV/readnaversid1750oid448aid0000195268.wav


MoviePy - Done.
MoviePy - Writing audio in ./WAV/readnaversid1750oid448aid0000195576.wav


MoviePy - Done.
MoviePy - Writing audio in ./WAV/readnaversid1750oid448aid0000196603.wav


MoviePy - Done.
MoviePy - Writing audio in ./WAV/video.wav


Using cache found in C:\Users\YangGang/.cache\torch\hub\pyannote_pyannote-audio_master                                 


MoviePy - Done.


,WAV_name,TimeStamp
0,./WAV/readnaversid1750oid448aid0000195268.wav,00:02:02.546000
0,./WAV/readnaversid1750oid448aid0000195576.wav,00:00:13.160000
0,./WAV/readnaversid1750oid448aid0000196603.wav,00:00:26.466000
0,./WAV/video.wav,00:00:19.104000


In [10]:
# 실행시간 결과
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

0:03:03
